In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import numpy as np
import torch
import hydra
import scipy.linalg as la
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from tqdm import tqdm, trange
from src.plot_utils import scatter_plot_2d
from src.utils import sparsity_measure
from omegaconf import OmegaConf
from torchvision.utils import make_grid
from pprint import pprint
from torch.utils.data import TensorDataset, DataLoader
from pathlib import Path
from datetime import datetime
from sklearn.decomposition import MiniBatchDictionaryLearning, SparseCoder, DictionaryLearning
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans

In [ ]:
sns.set_style('ticks')
sns.set_context("paper", font_scale=1.0, rc={"text.usetex": True})
sns.set_palette("colorblind")
sns.set_style({"font.family": "serif", "font.serif": ["Times New Roman"]})

In [ ]:
project_dir = Path('/n/home13/shubham/Current Projects/bioplausible_learning/code')

In [ ]:
with hydra.initialize(version_base=None, config_path="../../../configs"):
    cfg = hydra.compose(config_name='ksm_moons.yaml')
    exp_params = OmegaConf.to_container(cfg, resolve=True)
    

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

exp_params['device'] = 'device'

# figfolder = Path(f'../../results/simulated/highdim/{datetime.now().strftime("%m-%d")}/{datetime.now().strftime("%H-%M")}')
# os.makedirs(figfolder, exist_ok=True)    
pprint(exp_params)    

{'dataset': {'batch_size': 128, 'dim': 30, 'n_samples': 2500, 'name': 'moons'},
 'device': 'device',
 'model': {'K': 12,
           'Winit': 'Kmeans',
           'omega': 0.1,
           'perturbation': 0.1,
           'rho': 1.0,
           'sparsity': 0.125},
 'optimizer': {'Minit': 'zero',
               'latent_iters': 20,
               'log_interval': 5,
               'lrs': {'Z': 0.01,
                       'Z_decay': 0.9,
                       'interval': 20,
                       'params': 0.01},
               'max_epochs': 3000,
               'param_iters': 1,
               'vis_interval': 20}}


Simulated data generated from a tall dictionary

In [ ]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=exp_params['dataset']['n_samples'], noise=0.01)
fig, ax = plt.subplots(figsize=(5, 4))
ax.scatter(X[:, 0], X[:, 1], s=10, alpha=0.6)
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
ax.set_title('Input Data')
plt.tight_layout()
fig.show()

# Create pytorch dataset and dataloader
indices = torch.arange(X.shape[0])
dataset = TensorDataset(torch.from_numpy(X).float(), indices)
dataloader = DataLoader(dataset, batch_size=exp_params['dataset']['batch_size'], shuffle=True)

In [ ]:
# Random samples on each of the manifolds
np.random.seed(1)
samples_per_manifold = exp_params['model']['K'] // 2
print(samples_per_manifold)
manifold_samples = []
for i in range(2):
    idx = np.where(y == i)[0]
    samples = X[idx[np.random.choice(len(idx), samples_per_manifold)]]
    manifold_samples.append(samples)
    
manifold_samples = torch.from_numpy(np.vstack(manifold_samples)).float()
# Plot the samples
fig, ax = plt.subplots(figsize=(5, 4))
ax.scatter(X[:, 0], X[:, 1], s=10, alpha=0.4, c='orange')
# ax.scatter(manifold_samples[:, 0], manifold_samples[:, 1], s=100, c='r', marker='x', label='Initial atoms', linewidths=2)
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
ax.set_title('Input Data')
plt.tight_layout()
fig.savefig(results_folder / 'input_data.pdf', format='pdf')
fig.show()

In [175]:
W_init = manifold_samples

In [ ]:
# Compute Cluster centers for the given data
kmeans = KMeans(n_clusters=exp_params['model']['K'], random_state=42).fit(X)
W_init = kmeans.cluster_centers_
# W_init /= np.linalg.norm(W_init, axis=1, keepdims=True)

# sim_lam = 2*exp_params['model']['lam']
# print(sim_lam)
# coder = SparseCoder(dictionary=W_init, transform_alpha=exp_params['model']['lam'], transform_algorithm='lasso_lars')
# init_codes = coder.transform(X.numpy())

#### KSM on manifold learning

In [59]:
def compute_reconstruction_error(X, D, Z):
    """
    Compute the reconstruction error of the given data
    X: (n_samples, n_features)
    D: (n_components, n_features)
    Z: (n_samples, n_components)
    """
    prediction = Z @ D
    error = np.mean(np.linalg.norm(X - prediction, axis=1)**2)
    return error

In [177]:
from src.models import KSM_manifold
from src.optimizers import optimizationADMM_manifold


In [178]:
def initializeTensors(exp_params, n_samples, Zinit=None, Pinit=None, Minit=None, seed=42, device='cpu'):
    torch.manual_seed(seed)
    if Zinit is None:
        Z = torch.randn(size=(n_samples, exp_params['model']['K']), dtype=torch.float32, requires_grad=False)
    else:
        Z = Zinit.clone()
        
        
    if Pinit is None:    
        Z_t = Z.unsqueeze(-1)
        V = Z_t @ Z_t.transpose(-1, -2)
        P = V + exp_params['model']['omega'] * torch.diag_embed(Z_t.squeeze())
    else:
        P = Pinit.clone()
        
        
    if Minit is None:        
        M_t = torch.randn(size=P.shape, requires_grad=False, dtype=torch.float32)
        M = M_t @ M_t.transpose(-1, -2)
        M = M / torch.linalg.norm(M, dim=(1, 2), keepdims=True)
    else:
        M = Minit.clone()
        
        
    # print("Initialized M")
    
    variables = {'Z': Z, 'P': P}
    lagrange = {'M': M}
    
    return variables, lagrange

In [179]:
exp_params['device'] = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = exp_params['device']
# M_init = torch.zeros(size=(exp_params['dataset']['n_samples'], exp_params['model']['K'], exp_params['model']['K']), dtype=torch.float32, requires_grad=False).to(device)
# W = torch.from_numpy(W_init).float()
W = W_init
M_init = W @ W.T / 2


In [180]:
from sparsemax import Sparsemax
Ps = Sparsemax(dim=-1)

In [181]:
from src.utils import power_method_svd

In [182]:
M_init =  M_init.repeat(exp_params['dataset']['n_samples'], 1, 1).to(device)
# Z_init = torch.from_numpy(init_codes).float().to(device)
Z_init = torch.rand(size=(exp_params['dataset']['n_samples'], exp_params['model']['K']), dtype=torch.float32, requires_grad=False).to(device)
Z_init = Ps(Z_init)
variables, lagrange = initializeTensors(exp_params, n_samples=exp_params['dataset']['n_samples'], Zinit=Z_init, Minit=M_init, seed=42, device=device)
exp_params['optimizer']['max_epochs'] = 5000
exp_params['optimizer']['latent_iters'] = 15
exp_params['optimizer']['log_interval'] = 20
exp_params['optimizer']['lrs']['params'] = 2e-3
exp_params['optimizer']['param_iters'] = 1
exp_params['model']['omega'] = 0.1
model = KSM_manifold(exp_params, W_init=W)
eta = 1/ power_method_svd(model.W.T, device=device)
optimizer = optimizationADMM_manifold(exp_params, model, eta=eta)


In [183]:
current_path = Path.cwd()
results_folder = current_path / 'results'/f'{datetime.now().strftime("%m-%d")}/{datetime.now().strftime("%H-%M")}'
os.makedirs(results_folder, exist_ok=True)
from copy import deepcopy
cfg = deepcopy(exp_params)

# folder = project_dir / f'results/simulated/highdim/{datetime.now().strftime("%m-%d")}/{datetime.now().strftime("%H-%M")}'
os.makedirs(results_folder, exist_ok=True)
cfg.pop('device')
cfg = OmegaConf.create(cfg)
with open(results_folder / 'config.yaml', 'w') as f:
    OmegaConf.save(cfg, f)
    
pprint(exp_params)

{'dataset': {'batch_size': 128, 'dim': 30, 'n_samples': 2500, 'name': 'moons'},
 'device': device(type='cuda'),
 'model': {'K': 12,
           'Winit': 'Kmeans',
           'omega': 0.1,
           'perturbation': 0.1,
           'rho': 1.0,
           'sparsity': 0.125},
 'optimizer': {'Minit': 'zero',
               'latent_iters': 15,
               'log_interval': 20,
               'lrs': {'Z': 0.01,
                       'Z_decay': 0.9,
                       'interval': 20,
                       'params': 0.002},
               'max_epochs': 5000,
               'param_iters': 1,
               'vis_interval': 20}}


In [184]:
from src.admms import run_ADMM_kds_Ptrack

In [185]:
# true_vals = {'D': D, 'Z': Z}

loss_vals = run_ADMM_kds_Ptrack(model, optimizer, torch.from_numpy(X).float(), variables, lagrange, dataloader, exp_params,
                                device=device, update_eta=True)

Training:   0%|          | 0/5000 [00:00<?, ?it/s]

Training: 100%|██████████| 5000/5000 [29:39<00:00,  2.81it/s, Total Loss: -0.7329 | Similarity Loss: -0.7334 | Penalty: 0.0000 | R2_W: 0.9978 | R2_est: 0.9861 | D_sim: 0.0000 | Z_sim: 0.0000 | DW_sim: 0.0000 | new_eta: 0.0839]


In [ ]:
# Let's plot the losses
fig, axs = plt.subplots(1, 4, figsize=(20, 5))
iterations = np.arange(0, exp_params['optimizer']['max_epochs'], step=exp_params['optimizer']['log_interval'])
ax = axs[0]
ax.plot(iterations[1:], loss_vals['total_loss'][1:])
ax.set_title('Total Loss')
# ax.set_xscale('log')
ax.set_xlabel('Iterations')
ax.set_ylabel('Loss')

ax = axs[1]
ax.plot(iterations[1:], loss_vals['similarity_loss'][1:])
ax.set_title('Similarity Loss')
# ax.set_xscale('log')
ax.set_xlabel('Iterations')

# ax = axs[2]
# ax.plot(iterations, loss_vals['sparsity'])
# ax.set_title('Sparsity Measure')
# # ax.set_xscale('log')
# ax.set_xlabel('Iterations')

ax = axs[2]
ax.plot(iterations, loss_vals['penalty'])
ax.set_title('Penalty')
# ax.set_xscale('log')
ax.set_xlabel('Iterations')



# ax.plot(loss_vals['loss_Z'])


# Plot the r2 score and similarity measures
# fig, axs = plt.subplots(1, 2, figsize=(15, 5))
ax = axs[3]
# ax.plot(iterations[1:], loss_vals['r2_W'][1:], label=r'$R^2 (W)$')
ax.plot(iterations[1:], loss_vals['r2_est'][1:], label=r'$R^2$ (Estimated)')
# ax.set_xscale('log')
ax.set_title('R2 Score')
ax.set_xlabel('Iterations')
ax.set_ylabel('R2 Score')
ax.legend()

# ax = axs[1]
# ax.plot(iterations, loss_vals['dw_sim'], label='cos sim (W)')
# ax.plot(iterations, loss_vals['dest_sim'], label='cos sim (est)')
# # ax.set_xscale('log')
# ax.set_title('Cos Similarity for estimated dictionary')
# ax.set_xlabel('Iterations')
# ax.set_ylabel('Value')
# ax.legend()

# ax = axs[2]
# ax.plot(iterations, loss_vals['latent_sim'])
# # ax.set_xscale('log')
# ax.set_title('Cos Similarity for latent codes')
# ax.set_xlabel('Iterations')
# ax.set_ylabel('Value')

# fig.savefig(folder / 'r2_n_cossim.pdf', format='pdf', bbox_inches='tight')
fig.savefig(results_folder / 'loss_curves.pdf', format='pdf', bbox_inches='tight')

fig.show()


In [ ]:
# plot the data and reconstruction
fig, ax = plt.subplots(figsize=(6, 4))
D_W = model.W.detach().cpu().numpy() 
H_est = torch.mean(variables['P'], dim=0).detach().cpu()
Z_est = variables['Z'].detach().cpu()
D_est = (1 + model.omega) * torch.linalg.solve(H_est, Z_est.T) @ torch.from_numpy(X).float() / X.shape[0]
XhatW = Z_est @ D_W
Xhat = Z_est @ D_est
ax.scatter(X[:, 0], X[:, 1], s=10, alpha=0.4, label='Orginal Data', c='orange')
# ax.scatter(XhatW[:, 0], XhatW[:, 1], s=10, alpha=0.6, c='r', label='Reconstruction (W)')
ax.scatter(XhatW[:, 0], XhatW[:, 1], s=10, alpha=0.3, c='g', label='Reconstruction')
ax.scatter(D_est[:, 0], D_est[:, 1], s=100, c='r', marker='x', label='Estimated Dictionary', linewidths=3)
# ax.scatter(W_init[:, 0], W_init[:, 1], s=100, c='b', marker='X', label='Init Dictionary', linewidths=1)
# ax.scatter(D_est[:, 0], D_est[:, 1], s=100, c='y', marker='X', label='Estimated Dictionary (P)', linewidths=1)
ax.set_title('Manifold Reconstruction')
ax.legend()
fig.savefig(results_folder / 'moons.pdf', format='pdf', bbox_inches='tight')
fig.show()

In [ ]:
# Histogram of the support of the latent codes
fig, ax = plt.subplots(figsize=(6, 4))
# Compute the support of the latent codes
support = torch.sum((variables['Z'] > 1e-3).float(), dim=1)
# plot the histogram with border

sns.histplot(support.cpu().numpy(), bins=20, ax=ax, edgecolor='black', linewidth=1.2)
ax.set_title('Histogram of significant activations capture intrinsic dim.')
ax.set_xlabel('Support')
ax.set_ylabel('Frequency')
fig.savefig(results_folder / 'support_hist.pdf', format='pdf', bbox_inches='tight')
fig.show()

In [198]:
import pickle
with open(results_folder / 'results.pkl', 'wb') as f:
    pickle.dump({'loss_vals': loss_vals, 'model': model, 'variables': variables, 'lagrange': lagrange}, f)